In [1]:
import pymc3 as pm
import numpy as np

In [2]:
# Set number of data points
T = 1000

# Set true dimension of underlying phenomenon Z
k = 5

# Set dimensions of observations
d1 = 20
d2 = 30

In [3]:
# Generate lifting transformations
W1 = np.random.randn(d1, k)
W2 = np.random.randn(d2, k)

# Generate observation means
mu1 = np.random.randn(d1, 1)
mu2 = np.random.randn(d2, 1)

# Generate observation covariance; should sample these from LKJ
pre_Psi1 = np.random.randn(2 * d1, d1)
Psi1 = np.dot(pre_Psi1.T, pre_Psi1)
pre_Psi2 = np.random.randn(2 * d2, d2)
Psi2 = np.dot(pre_Psi2.T, pre_Psi2)

In [4]:
# Compute 'standard deviation' matrices
Psi1_sqrt = np.linalg.cholesky(Psi1)
Psi2_sqrt = np.linalg.cholesky(Psi2)

In [5]:
# Generate underlying phenomenon Z
Z = np.random.randn(T, k)

# Use lifting transformations to get distorted Zs
Z_lift1 = np.dot(W1, Z.T).T
Z_lift2 = np.dot(W2, Z.T).T

# Shift Z by mean of observations
Z_shift1 = Z_lift1 + mu1.T
Z_shift2 = Z_lift2 + mu2.T

# Generate white noise for Xs
X1_noise = np.random.randn(T, d1)
X2_noise = np.random.randn(T, d2)

# Scale and shift noise to match observation distributions
X1 = np.dot(X1_noise, Psi1_sqrt) + Z_shift1
X2 = np.dot(X2_noise, Psi2_sqrt) + Z_shift2

In [ ]:
model = pm.Model()

with model:
    # Priors on Z-lifting matrices
    W1_var = pm.Normal('W1', shape=(d1,k))
    W2_var = pm.Normal('W2', shape=(d2,k))
    
    # Priors on X means
    mu1_var = pm.Normal('mu1', shape=(d1,1))
    mu2_var = pm.Normal('mu2', shape=(d2,1))
    
    # Priors on covariance matrices
    Psi1_var = pm.LKJCholeskyCov(
        'Psi1', n=d1, eta=10.0 * d1, sd_dist=pm.HalfCauchy.dist(2.5))
    Psi2_var = pm.LKJCholeskyCov(
        'Psi2', n=d2, eta=10.0 * d2, sd_dist=pm.HalfCauchy.dist(2.5))
    
    # SDs for the Xs
    L1 = pm.expand_packed_triangular(d1, Psi1_var)
    L2 = pm.expand_packed_triangular(d2, Psi2_var)
    
    # Likelihood on Z
    Z_var = pm.MvNormal('Z', mu=np.zeros(k), chol=np.eye(k), shape=(T,k))
    
    # Means for conditional likelihood on Xs conditioned on Z
    Z_lift1_var = pm.Deterministic('Z_lif1', W1_var.dot(Z_var.T).T)
    Z_lift2_var = pm.Deterministic('Z_lif2', W2_var.dot(Z_var.T).T)
    Z_shift1_var = pm.Deterministic('Z_shift1', Z_lift1_var + mu1_var.T)
    Z_shift2_var = pm.Deterministic('Z_lift2', Z_lift2_var + mu2_var.T)
    
    # Conditional likelihoods of Xs conditioned on Z
    X1_var = pm.MvNormal('X1', mu=Z_shift1_var, chol=L1, observed=X1)
    X2_var = pm.MvNormal('X2', mu=Z_shift2_var, chol=L2, observed=X2)
    
    pm.sample(live_plot=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Z, Psi2_cholesky_cov_packed__, Psi1_cholesky_cov_packed__, mu2, mu1, W2, W1]
  9%|▉         | 93/1000 [00:20<03:15,  4.65it/s]